In [ ]:
from langchain_community.utilities import SQLDatabase
from pyprojroot import here
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path='../.env')

Connecting to the sqldb

In [ ]:
db_path = r"data\sql\bank_domain.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
db

In [ ]:
# validate the connection to the vectordb
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM accounts LIMIT 10;")

In [ ]:
from langchain_groq import ChatGroq
# llm=ChatGroq(model="llama3-70b-8192")
llm = ChatGroq(
    temperature = 0,
    groq_api_key = os.getenv("groq_api_key"),
    model_name = os.getenv("llama_model_name")
)

In [ ]:
llm.invoke("hello how are you?")

#### 1. SQL query chain

In [ ]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
print(response)

In [ ]:
db.run("SELECT COUNT('EmployeeId') FROM 'Employee';")

In [ ]:
chain.get_prompts()[0].pretty_print()

#### Add QuerySQLDataBaseTool to the chain

In [ ]:
from langchain_community.tools import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

chain.invoke({"question": "How many employees are there"})

#### Answer the question in a user friendly manner

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

#### 2. Agents

In [ ]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", 
                                  verbose=True,
                                  agent_executor_kwargs = {"return_intermediate_steps": True}
                                  )

In [ ]:
response = agent_executor.invoke(
    {
        "input": "What is the average balance of accounts opened in the last six months, categorized by account type and broken down by the state of the branch the associated customer is linked to, via employee records?"
    }
)

In [ ]:
queries = []
for (log, output) in response["intermediate_steps"]:
    if log.tool == 'sql_db_query':
        queries.append(log.tool_input)

In [ ]:
print(f"Query\n{queries[-1]['query']}\nAnswer\n{response['output']}")

In [ ]:
response['output']